<a href="https://colab.research.google.com/github/debaggi/Junior-Data-Analyst-Cases/blob/main/SamokatTech_Jinior_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Привет!

Всего 3 задания

----------
1 задание
----------

Нужно на основе приложенных выгрузок сделать отчет в Power BI и отправить файл .pbix
(если файл не влезает в почту - положи его на гугл диск и пошарить ссылкой)
Обратно слать выгрузки не нужно.

Требования к отчету - представь, что это все данные, которые вообще есть и ты делаешь первый в компании отчет.
Твоя задача выжать из этих выгрузок максимум пользы.
Какие показатели посчитать, какие визуализации выбрать и как организовать отчет - твой выбор.

Выгрузки:
- warehouses
- products
- orders
- order_lines

В выгрузках рандомные данные, но стоит воспринимать их как фактические.
Всю обработку нужно делать внутри Power BI - исходные файлы оставь как есть.

Бонус:
Если найдешь что-то интересное (аномалии, тренды, зависимости, etc.) - оформи свои мысли в ворде,
сопроводив скринами визуализаций из Power BI и отправь файл .docx
Если хочется сделать в формате презентации или описать внутри Power BI - тоже гуд.

----------
2 задание
----------

Выгрузить список user_id тех пользователей,
кто купил за период 1-15 августа 2 любых корма для животных, кроме "Корм Kitekat для кошек, с кроликом в соусе, 85 г".
Отправить файл .xls

----------
3 задание
----------

Напиши SQL запрос к воображаемой БД, которая имеет структуру как у тебя получилось в 1 задании в Power BI.
Т.е. те же 4 таблицы, те же ключи и те же поля.
Запрос должен вернуть топ 5 самых часто встречающихся товаров в первых заказах пользователей в СПб за период 15-30 августа.

--
После того как сделаешь - отправляй на почту HR.

Удачи :)

https://pavelbukhtik.notion.site/21d3e6b006284d189514e517d74fa369

In [ ]:
import pandas as pd

In [ ]:
from  google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_dir = '/content/content/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/'
path_files = ['/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/orders.csv', '/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/products.csv', '/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/warehouses.csv', '/content/drive/MyDrive/Technical Interview/Samokat Tech Data Analytics: Jun.Grade/order_lines.txt']

orders = pd.read_csv(path_files[0])
orders = orders.set_index('warehouse_id')

warehouses = pd.read_csv(path_files[2])

display(orders.head(3))
display(warehouses.head(3))

Цели:
<!-- клиенты -->
- определить количество заказов по каждому даркстору
- определить топ-3 id клиентов, которые наиболее часто осущесвляют заказы
<!-- дарксторы -->
- определить наиболее и наименее часто используемые дарксторы
- определить города, в которых расположены дарксторы из топ-3 дарксторов


In [ ]:
# Внутреннее соединение таблицы orders и таблицы warehouses по атрибуту wirehouse_id
warehouses_orders = pd.merge(orders, warehouses, on='warehouse_id', how='inner')
warehouses_orders['order_date'] = pd.to_datetime(warehouses_orders['order_date'])

print('Сгруппированная таблица заказов и таблица дарксторов по атрибуту wirehouse_id\n')
warehouses_orders.sort_values('warehouse_id').head(3)

In [ ]:
# записываю в csv-файл полученную таблицу
warehouses_orders.to_csv('warehouses_orders.csv')

In [ ]:
# warehouses_orders = pd.read_csv('warehouses_orders.csv')

In [ ]:
#таблица warehouses_orders сгруппирована атрибуту warehose_id и отсортирована по убыванию параметра count_orders
warehouses_orders_grouped = warehouses_orders.groupby(['warehouse_id'])[['order_id']].agg('count').rename(columns={'order_id': 'count_orders'}).sort_values('count_orders', ascending=False)
warehouses_orders_grouped.iloc[:3]

### Анализ дарксторов

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 4))
sns.lineplot(y='count_orders', x='warehouse_id', data=warehouses_orders_grouped, marker='.', markersize=15/2, color='green', alpha=.6)
plt.title('The numbers of orders in the context of darkstores', font='Monospace')
plt.xticks(font='Monospace')
plt.yticks(font='Monospace');

In [ ]:
# вычисление суммарного числа заказов в разрезе дарксторов
warehouses_top_3_max = list(warehouses_orders_grouped.iloc[:3].index.values) # топ-3-winners дарксторов по числу заказов
warehouses_top_3_min = list(warehouses_orders_grouped.iloc[-3:].index.values) # топ-3-losers дарксторов по числу заказов

# вычисление суммарного числа заказов по всем дарксторам за все время
total_sum_purchases = warehouses_orders['order_id'].count()

# процент заказов, который приходится на даркстор
warehouses_orders_grouped['percentage_orders'] = round(warehouses_orders_grouped['count_orders']/total_sum_purchases * 100, 2)
warehouses_orders_grouped.to_excel('warehouses_orders.xlsx', sheet_name='warehouses_orders_grouped')


print(f'Суммарное количество заказов по всем дарксторам сети за все время: {total_sum_purchases}')
print('\n')
print('----------------------------------------------------')
print(f'Дарксторы с максимальным числом заказов: {warehouses_top_3_max}')
print(display(warehouses_orders_grouped.head(3)))
print('\n')
print('----------------------------------------------------')
print(f'Дарксторы с минимальным числом заказов: {warehouses_top_3_min}')
print(display(warehouses_orders_grouped.iloc[-3:].head(3)))
print('----------------------------------------------------')

In [ ]:
# информация о топ-3 дарксторах по числу заказов за все время
info_warehouses_top_3_max = warehouses[warehouses['warehouse_id'].isin(warehouses_top_3_max)]
display(info_warehouses_top_3_max.set_index('warehouse_id'))
print('\n')
info_warehouses_top_3_min = warehouses[warehouses['warehouse_id'].isin(warehouses_top_3_min)]
display(info_warehouses_top_3_min.set_index('warehouse_id'))

### Анализ клиентов

In [ ]:
# информация о топ-3 покупателях по числу заказов за все время
top_3_clients = pd.DataFrame(warehouses_orders.groupby('user_id')['order_id'].agg('count')).rename(columns={'order_id':'count_orders'})
top_3_clients_sorted = top_3_clients.sort_values('count_orders', ascending=False)
list_top_3_clients_id = list(top_3_clients_sorted.index[:3].values)

print(f'Клиенты, входящие к категорию топ-3 клиентов по числу заказов за все время: {list_top_3_clients_id}')

In [ ]:
# получить из таблицы users данные о  топ-3-клиентах
# users

In [ ]:
# фильтрация таблицы warehouses_orders по атрибуту user_id
info_orders_by_top_3_clients = warehouses_orders[warehouses_orders['user_id'].isin(list_top_3_clients_id)][['user_id', 'order_date', 'warehouse_id']].set_index('user_id')
display(info_orders_by_top_3_clients.sort_values('order_date'))

In [ ]:
# Вопрос: каковы даты заказов топ-3 клиентов

info_orders_by_top_3_clients['year'] = info_orders_by_top_3_clients['order_date'].dt.year
info_orders_by_top_3_clients['month'] = info_orders_by_top_3_clients['order_date'].dt.month
info_orders_by_top_3_clients['day'] = info_orders_by_top_3_clients['order_date'].dt.day

info_orders_by_top_3_clients_short = info_orders_by_top_3_clients[['year', 'day', 'month', 'warehouse_id']]
info_orders_by_top_3_clients_short_pivot = pd.pivot_table(info_orders_by_top_3_clients_short, index=['user_id', 'year', 'month', 'day', 'warehouse_id'])

print(f'Сводная таблица "Сведения о заказах топ-3 клиентов" \n{display(info_orders_by_top_3_clients_short_pivot)}')

In [ ]:
info_orders_by_top_3_clients_short_pivot.to_csv('info_orders_by_top_3_clients_short_pivot.csv')

## Dashboard

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# График распределения процентов заказов в разрезе дарксторов
fig, ax = plt.subplots(figsize=(21, 7), nrows=2, ncols=2)
ax[0,0].set_title('top-3 warehouses')
sns.barplot(x='warehouse_id', y='percentage_orders', data=warehouses_orders_grouped.head(3), color='green', alpha=0.6, ax=ax[0,0])

### Перекодировка файла с данными

In [ ]:
import chardet

In [ ]:
with open(path_files[1], 'rb') as raw_file:
  output_file = chardet.detect(raw_file.read(10000))
  encoding = output_file['encoding']

with open(path_files[1], 'r', encoding=encoding) as dataset:
  products_content = dataset.read()